In [2]:
import numpy as np
import pandas as pd
import json
from geopy.distance import great_circle

Read json file contains border points coordinates:

In [3]:
with open("Border_Points.json", "r") as file:
    data = json.load(file)

In [4]:
data = data["features"][0]["geometry"]["coordinates"][0]
len(data)

2284

To obtain destribution of points in each of four coordinate regions:

In [5]:
Sari_coor   = [36.561151, 53.060088] # representing a point separates Iranian border into eastern and wertern regions
Ilam_coor = [33.568375, 46.045389]  # represinting a point separates Iranian border into northern and southern regions

In [6]:
Ilam_lat = Ilam_coor[0]
Ilam_lng = Ilam_coor[1]
Sari_lat = Sari_coor[0]
Sari_lng = Sari_coor[1]

In [7]:
points = pd.DataFrame(data, columns=["lng", "lat"])
points.head()

,lng,lat
0,48.613949,29.928497
1,48.621193,29.934804
2,48.624422,29.934589
3,48.630881,29.932651
4,48.640818,29.932005


In [8]:
len_NE = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] > Sari_lng)])
len_NW = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] < Sari_lng)])
len_SW = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] < Sari_lng)])
len_SE = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] > Sari_lng)])
print(f"len_NE : {len_NE}")
print(f"len_NW : {len_NW}")
print(f"len_SW : {len_SW}")
print(f"len_SE : {len_SE}")

len_NE : 94
len_NW : 446
len_SW : 768
len_SE : 976


To complete points in NE region:

In [9]:
with open("NE_Complete.json", "r") as file:
    NE_Completoin_data = json.load(file)

In [10]:
NE_Completion = list()

for feature in NE_Completoin_data["features"]:
    NE_Completion.append(feature["geometry"]["coordinates"][:])

NE_Completion = [element for sublist in NE_Completion for element in sublist]

In [11]:
NE_Completion = pd.DataFrame(NE_Completion, columns=["lng", "lat"])

In [12]:
len(NE_Completion)

923

In [13]:
NE_Completion.head()

,lng,lat
0,60.551479,33.640504
1,60.551331,33.643335
2,60.551035,33.645919
3,60.551183,33.648627
4,60.551183,33.651580


To complete points in NW region:

In [14]:
with open("NW_Complete.json", "r") as file:
    NW_Completoin_data = json.load(file)

In [15]:
NW_Completion = list()

for feature in NW_Completoin_data["features"]:
    NW_Completion.append(feature["geometry"]["coordinates"][:])

NW_Completion = [element for sublist in NW_Completion for element in sublist]

In [16]:
NW_Completion = pd.DataFrame(NW_Completion, columns=["lng", "lat"])

In [17]:
len(NW_Completion)

552

In [18]:
NW_Completion.head()

,lng,lat
0,45.924524,33.598682
1,45.912369,33.620953
2,45.902645,33.641194
3,45.844300,33.627025
4,45.810265,33.610830


To complete points in SW region:

In [19]:
with open("SW_Complete.json", "r") as file:
    SW_Completoin_data = json.load(file)

In [20]:
SW_Completion = list()

for feature in SW_Completoin_data["features"]:
    SW_Completion.append(feature["geometry"]["coordinates"][:])

SW_Completion = [element for sublist in SW_Completion for element in sublist]

In [21]:
SW_Completion = pd.DataFrame(SW_Completion, columns=["lng", "lat"])

In [22]:
len(SW_Completion)

216

In [23]:
SW_Completion.head()

,lng,lat
0,54.007617,26.755803
1,53.954357,26.735419
2,53.885879,26.708235
3,53.802184,26.721828
4,53.741315,26.701438


To merge dataframes:

In [24]:
points = pd.concat([points, NE_Completion, NW_Completion, SW_Completion], ignore_index=True)
points

,lng,lat
0,48.613949,29.928497
1,48.621193,29.934804
2,48.624422,29.934589
3,48.630881,29.932651
4,48.640818,29.932005
...,...,...
3970,46.169003,33.069856
3971,45.866036,33.496451
3972,45.946185,33.548421
3973,45.913532,33.630024


Obtain destribution of points in each of four coordinate regions one more time:

In [25]:
len_NE = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] > Sari_lng)])
len_NW = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] < Sari_lng)])
len_SW = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] < Sari_lng)])
len_SE = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] > Sari_lng)])
print(f"len_NE : {len_NE}")
print(f"len_NW : {len_NW}")
print(f"len_SW : {len_SW}")
print(f"len_SE : {len_SE}")

len_NE : 1017
len_NW : 1000
len_SW : 963
len_SE : 995


Optimization and searching for the exact center of Iran:

In [66]:
lng_min = 49.781853529617024
lng_max = 58.37602322566812
lat_min = 29.235933551806966
lat_max = 35.393730146031245

In [67]:
points_copy = points.copy()

In [68]:
center_candidates = []
lat_candidate = 0
lng_candidate = 0

In [69]:
step = 5
min_variance = 145231.6822812455

while step > 0.00000000000001:
    print(step)
    for lng in np.arange(lng_min, lng_max, step):
        for lat in np.arange(lat_min, lat_max, step):
            points["distances"] = points.apply(lambda row: great_circle((row["lat"], row["lng"]), (lat, lng)).kilometers, axis=1)        # calculate variance
            variance = points["distances"].var(ddof=0)
            if variance < min_variance:
                min_variance = variance
                lat_candidate = lat
                lng_candidate = lng

    center_candidates.append([lat_candidate, lng_candidate])
    # Lowering search area:                
    lat_min = lat_candidate - step
    lat_max = lat_candidate + step
    lng_min = lng_candidate - step
    lng_max = lng_candidate + step
    step /= 10

    print(lat_candidate)
    print(lng_candidate)
    print(7*"=")


5
34.235933551806966
54.781853529617024
0.5
31.735933551806966
54.781853529617024
0.05
31.835933551806974
54.731853529617
0.005
31.830933551806964
54.73685352961703
0.0005
31.83193355180695
54.73835352961706
5e-05
31.831883551806968
54.738453529617075
5e-06
31.831893551806985
54.73842852961708
5.000000000000001e-07
31.831892051806975
54.738429529617065
5.000000000000001e-08
31.83189205180698
54.73842942961704
5.000000000000001e-09
31.831892051806985
54.73842941961702
5.000000000000001e-10
31.831892051806985
54.73842941961702
5.0000000000000015e-11
31.831892051806985
54.73842941961702
5.000000000000001e-12
31.831892051806985
54.73842941961702
5.000000000000001e-13
31.831892051806985
54.73842941961702
5.000000000000001e-14
31.831892051806985
54.73842941961702


In [70]:
center_candidates

[[34.235933551806966, 54.781853529617024],
 [31.735933551806966, 54.781853529617024],
 [31.835933551806974, 54.731853529617],
 [31.830933551806964, 54.73685352961703],
 [31.83193355180695, 54.73835352961706],
 [31.831883551806968, 54.738453529617075],
 [31.831893551806985, 54.73842852961708],
 [31.831892051806975, 54.738429529617065],
 [31.83189205180698, 54.73842942961704],
 [31.831892051806985, 54.73842941961702],
 [31.831892051806985, 54.73842941961702],
 [31.831892051806985, 54.73842941961702],
 [31.831892051806985, 54.73842941961702],
 [31.831892051806985, 54.73842941961702],
 [31.831892051806985, 54.73842941961702]]